In [1]:
def parse_file(file_path):
    data = {}
    torso_diameters = []

    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        try:
            parts = [part.strip() for part in line.strip().split(',')]
            video = parts[0].split(': ')[1]
            frame = parts[1].split(': ')[1]
            keypoint = parts[2].split(' ')[1]
            euclidean_distance = float(parts[3].split(' = ')[1])
            torso_diameter = float(parts[4].split(' = ')[1])
            
          
            torso_diameters.append(torso_diameter)

            key = f"{video},{frame},{keypoint}"
            data[key] = (euclidean_distance, torso_diameter)

        except IndexError:
            print(f"Error parsing line: {line}")
            continue

    min_diameter = min(torso_diameters)
    max_diameter = max(torso_diameters)
    avg_diameter = sum(torso_diameters) / len(torso_diameters)

    return data, min_diameter, max_diameter, avg_diameter



def calculate_pdj(data):
    total_joints = len(data)
    correct_joints = sum(1 for value in data.values() if value[0] <= 0.2 * value[1])
    pdj = (correct_joints / total_joints) * 100 if total_joints > 0 else 0.0
    return pdj


file_path_model1 = 'mediapipe_keypoint_distances.txt'
file_path_model2 = 'movenet_keypoint_distances.txt'


data_model1, min_dia1, max_dia1, avg_dia1 = parse_file(file_path_model1)
data_model2, min_dia2, max_dia2, avg_dia2 = parse_file(file_path_model2)




matching_keys = set(data_model1.keys()) & set(data_model2.keys())

print(f"Number of keys in Model 1: {len(data_model1)}")
print(f"Number of keys in Model 2: {len(data_model2)}")
print(f"Number of matching keys: {len(matching_keys)}")

matching_data_model1 = {key: data_model1[key] for key in matching_keys}
matching_data_model2 = {key: data_model2[key] for key in matching_keys}


pdj_model1 = calculate_pdj(data_model1)
pdj_model2 = calculate_pdj(data_model2)


pdj_model1_m = calculate_pdj(matching_data_model1)
pdj_model2_m = calculate_pdj(matching_data_model2)

print(f"Model 1 (Mediapipe) - Min Torso Diameter: {min_dia1}, Max Torso Diameter: {max_dia1}, Avg Torso Diameter: {avg_dia1}")
print(f"Model 2 (Movenet) - Min Torso Diameter: {min_dia2}, Max Torso Diameter: {max_dia2}, Avg Torso Diameter: {avg_dia2}")


print(f'Mediapipe PDJ: {pdj_model1:.2f}%')
print(f'Movenet PDJ: {pdj_model2:.2f}%')

print(f'Mediapipe PDJ for matching entries: {pdj_model1_m:.2f}%')
print(f'Movenet PDJ for matching entries: {pdj_model2_m:.2f}%')



Number of keys in Model 1: 2606800
Number of keys in Model 2: 2838402
Number of matching keys: 2606800
Model 1 (Mediapipe) - Min Torso Diameter: 10.0, Max Torso Diameter: 69.26037828369117, Avg Torso Diameter: 40.900559750771365
Model 2 (Movenet) - Min Torso Diameter: 10.0, Max Torso Diameter: 69.26037828369117, Avg Torso Diameter: 40.111385446011134
Mediapipe PDJ: 80.89%
Movenet PDJ: 70.63%
Mediapipe PDJ for matching entries: 80.89%
Movenet PDJ for matching entries: 75.98%
